# Calculate Risk categories from the multilayer perceptron model pooled results

In [1]:
!pip install --upgrade pip
!pip install pandas

In [3]:
import numpy as np
import pandas as pd
import pickle

import sys
#path to the functions folder
sys.path.append('../onc_functions')

from risk import get_risk_categories

print('python-' + sys.version)
import datetime
dte = datetime.datetime.now()
dte = dte.strftime("%Y%m%d")

python-3.9.5 (v3.9.5:0a7dcbdb13, May  3 2021, 13:17:02) 
[Clang 6.0 (clang-600.0.57)]


In [2]:
# here we import the dataset we created for the fairness assessment in notebook 6
with open('complete_fair1.pickle','rb') as f:  
    details = pickle.load(f)
details.head()

,usrds_id,died_in_90,inc_age,sex,dialtyp,race,hispanic,subset,impnum
0,2969676.0,0.0,72.0,2.0,1.0,2.0,2.0,8,1
1,2969869.0,1.0,73.0,2.0,1.0,2.0,2.0,9,1
2,2970021.0,0.0,55.0,2.0,1.0,1.0,1.0,8,1
3,2970067.0,0.0,72.0,1.0,1.0,1.0,2.0,9,1
4,2970686.0,0.0,49.0,2.0,1.0,2.0,2.0,8,1


In [3]:
with open('./results/2021_final_MLP_model_test_pred_proba_pooled.pickle','rb') as f:  
    y_pred = pickle.load(f)
y_pred.head()

,imp1,imp2,imp3,imp4,imp5,score,std_,usrds_id,died_in_90,subset
0,0.677572,0.764768,0.582416,0.796379,0.646979,0.693623,0.078001,31089.0,1.0,9
1,0.657155,0.718089,0.527093,0.748711,0.641911,0.658592,0.076464,34521.0,0.0,8
2,0.457672,0.350533,0.513291,0.560138,0.505401,0.477407,0.071277,46751.0,0.0,8
3,0.515523,0.605403,0.402248,0.716361,0.562594,0.560426,0.103325,50506.0,0.0,9
4,0.165916,0.199461,0.176441,0.313378,0.225053,0.216050,0.052747,54985.0,0.0,8


In [4]:
data = details.merge(y_pred, on=['usrds_id','died_in_90','subset'])
data.head()

,usrds_id,died_in_90,inc_age,sex,dialtyp,race,hispanic,subset,impnum,imp1,imp2,imp3,imp4,imp5,score,std_
0,2969676.0,0.0,72.0,2.0,1.0,2.0,2.0,8,1,0.464458,0.536613,0.481625,0.516356,0.412390,0.482288,0.043164
1,2969869.0,1.0,73.0,2.0,1.0,2.0,2.0,9,1,0.513067,0.574948,0.510624,0.484547,0.526900,0.522017,0.029797
2,2970021.0,0.0,55.0,2.0,1.0,1.0,1.0,8,1,0.460098,0.527486,0.487378,0.472566,0.557690,0.501043,0.036283
3,2970067.0,0.0,72.0,1.0,1.0,1.0,2.0,9,1,0.054770,0.050509,0.049944,0.046526,0.054721,0.051294,0.003130
4,2970686.0,0.0,49.0,2.0,1.0,2.0,2.0,8,1,0.163485,0.193392,0.167893,0.117967,0.149734,0.158494,0.024699


In [5]:
risk_cat = get_risk_categories(data,
                               y_proba_col_name='score',
                               y_true_col_name='died_in_90')
risk_cat

,Risk Category,Count,Count Died in 90,Count Survived,Percent Died in 90
0,0-0.09,57014,245.0,56769.0,0.004297
1,0.1-0.19,66582,928.0,65654.0,0.013938
2,0.2-0.29,47796,1359.0,46437.0,0.028433
3,0.3-0.39,37752,1791.0,35961.0,0.047441
4,0.4-0.49,33388,2423.0,30965.0,0.072571
5,0.5-0.59,30980,3234.0,27746.0,0.104390
6,0.6-0.69,30237,4652.0,25585.0,0.153851
7,0.7-0.79,29631,6861.0,22770.0,0.231548
8,0.8-0.89,11472,4036.0,7436.0,0.351813
9,0.9-0.99,453,209.0,244.0,0.461369


In [6]:
risk_cat.to_csv('./results/' + str(dte) + '_mlp_risk_cat.csv')